### Toy problem to count all words in the dataset in order to test reading the dataset

In [ ]:
from datasets import load_dataset
import nltk
import itertools
from collections import Counter
from tqdm import tqdm
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
import re

In [ ]:
# deze cel downloadet de dataset
# hij gebruikt het script cnn_dailymail.py van huggingface:
# https://huggingface.co/datasets/cnn_dailymail/tree/main
# dit kan wel iets van 10 minuten duren, ga maar wat koffie halen

dataset = load_dataset("cnn_dailymail.py", "3.0.0", split="train") 

In [ ]:
# print de features en het aantal datapunten in de dataset
print(dataset.features)
print(dataset.num_rows)

In [ ]:
# methoden om de data te pre-processen
en_stemmer = EnglishStemmer() # stemmer voor engelse woorden
nltk.download('stopwords') # stopwoorden die niet veel waarde toevoegen
stop_words = set(stopwords.words('english'))
alph_string_pattern = re.compile("[a-zA-Z]") # filtert 'woorden' die niet beginnen met een letter, zoals interpunctietokens


def word_counter_text(text: str, stem=False, remove_stopwords=False):
    """
    Neemt als input een string tekst
    Returnt een Counter object die alle woorden uit de tekst telt
    """
    
    # splits de tekst op in een lijst van woorden
    sents = nltk.tokenize.sent_tokenize(text)
    words = [nltk.word_tokenize(sent) for sent in sents]
    flatten_words = list(itertools.chain(*words))
    
    # woorden stemmen of alleen maar hoofdletters weghalen
    if stem:
        flatten_lower_words = [en_stemmer.stem(str) for str in flatten_words]
    else:
        flatten_lower_words = [str.lower() for str in flatten_words]
    
    # stopwoorden weghalen
    if remove_stopwords:
        flatten_lower_words = [str for str in flatten_lower_words if str not in stop_words]
        
    # tokens die niet beginnen met een letter weghalen
    flatten_lower_words = [str for str in flatten_lower_words if alph_string_pattern.match(str)]
    
    return Counter(flatten_lower_words)

In [ ]:
texts = dataset['article']

words = Counter() # Counter object dat alle woorden telt
for i in tqdm(range(2000)): # kijkt alleen naar de eerste 2000 artikelen, anders duurt het een half uur
    words += word_counter_text(texts[i], stem=True, remove_stopwords=True) # tel de Counters bij elkaar op voor elk artikel

In [ ]:
print(words.most_common(50))
print(len(words.items()), 'verschillende woorden')